In [1]:
import sys
import numpy as np
#from subcipher import *
from math import gcd
from collections import Counter

In [ ]:
def c2i(c, alphabet):
    """Returns the index of c in the string alphabet, starting at 0"""
    return alphabet.index(c)

def i2c(i, alphabet):
    """Returns the character at index i in alphabet"""
    return alphabet[i]

def prepare_string(s, alphabet):
    """removes characters from s not in alphabet, returns new string"""
    s = s.upper()
    finalString = ""
    for i in s:
      if i in alphabet:
        finalString += i
    return finalString
    
def last_index_of(array, string):
    i = len(array) - 1
    while i > 0:
      if array[i] == string:
        return i
      i = i - 1
    
    return 0
    
def vigenere_encode(plaintext, key, alphabet):
    """Returns plaintext encoded by vigenere cipher using key"""
    plaintext = plaintext.upper()
    plaintext = prepare_string(plaintext, alphabet)
    
    finalString = ""
    
    i = 0
    for char in plaintext:
      stringnum = c2i(key[i % len(key)], alphabet)
      plaintextnum = c2i(char, alphabet)
      finalString += i2c((stringnum + plaintextnum) % len(alphabet), alphabet)
      i+=1
    
    return finalString


def vigenere_decode(ciphertext, key, alphabet):
    """Returns ciphertext decoded by vigenere cipher using key"""
    ciphertext = ciphertext.upper()
    ciphertext = prepare_string(ciphertext, alphabet)
    
    finalString = ""
    
    i = 0
    for char in ciphertext:
      stringnum = c2i(key[i % len(key)], alphabet)
      ciphertextnum = c2i(char, alphabet)
      finalString += i2c((ciphertextnum - stringnum + 26) % len(alphabet), alphabet)
      i+=1
    
    return finalString
  
def index_of_coincidence(ciphertext, alpha):
    common = Counter(ciphertext)
    ioc = 0

    for index in list(alpha):
      ioc = ioc + (common[index] * (common[index] - 1))
      
    ioc = ioc * 1/len(ciphertext) * 1/(len(ciphertext) - 1)
    return ioc

def friedman_test(ciphertext, alpha):
    l = 0
    
    n = len(ciphertext)
    i = index_of_coincidence(ciphertext, alpha)
    l = n * (0.027)/((n-1)*i + 0.0655 - 0.0385 * n)
    
    return l

def kasiski_test(ciphertext):  #Code partially provided
    """Finds gcd of most common distances between repeated trigraphs
    Recommended strategy: loop through the ciphertext, keeping a list of trigraphs and a list of distances in this way:
    1) When encountering a new trigraph add it to the trigraph list
    2) When encountering a repeat add the distance from current index to first index of that trigraph to the list of distances"""

    trigraphs = []
    distances = []
    
    i = 0
    for i in range(len(ciphertext) - 2):
      newgraph = ciphertext[i:i+3]
      if newgraph not in trigraphs:
        trigraphs.append(newgraph)
        trigraphs.append(i)
      else:
        lastindex = last_index_of(trigraphs, newgraph)
        distances.append(i - (trigraphs[lastindex + 1]))
        
        trigraphs.append(newgraph)
        trigraphs.append(i)
      i += 1
        

    # Code is provided to find the gcd of any common distances appearing at least twice, just add your array:
    dCount = Counter(distances)
    topCount = dCount.most_common(6)
    my_gcd = topCount[0][0]
    for index in range(1, len(topCount)):
        if topCount[index][1] > 1:
            my_gcd = gcd(my_gcd, topCount[index][0])
    return my_gcd 

def run_key_tests(ciphertext, alphabet): #Code provided
    """Runs Friedman and Kasiski tests and formats the output nicely"""
    friedman = friedman_test(ciphertext, alphabet)
    kasiski = kasiski_test(ciphertext)
    out = "Friedman test gives: " + str(friedman) + " and Kasiski gives this as the most likely: " + str(kasiski);
    return out

def make_cosets(text, n):
    cosets = []
  
    for index in range(n):
      i = 0
      nextword = ""
      while (i + index) <= (len(text) - 1):
        nextword += text[i + index]
        i += n
      
      cosets.append(nextword)
    
    return cosets
    """Makes cosets out of a ciphertext given a key length; should return an array of strings"""

def rotate_list(old_list):  #Code provided
    """Takes the given list, removes the first element, and appends it to the end of the list, then returns the
    new list"""
    new_list = old_list[:]
    new_list.append(old_list[0])
    del new_list[0]
    return new_list

def find_total_difference(list1, list2):
    """Takes two lists of equal length containing numbers, finds the difference between each pair of matching
    numbers, sums those differences, and returns that sum"""
    
    finalSum = 0
    i = 0
    while i < len(list1):
      finalSum += abs(list2[i] - list1[i])
      i += 1
    
    return finalSum  
    
def find_likely_letters(coset, alpha, eng_freq):
    """Finds the most likely shifts for each coset and prints them
    Recommended strategy: make a list of the frequencies of each letter in the coset, in order, A to Z.
    Then, alternate using the find total difference method (on your frequencies list and the standard english
    frequencies list) and the rotate list method to fill out a new list of differences.  This makes a list of
    the total difference for each possible encryption letter, A to Z, in order.
    Then, find the indices of the smallest values in the new list, and i2c them for the most likely letters."""
    
    coset_freq = []
    differences = []
    
    for index in alpha:
      i = 0
      number = 0
      while i < len(coset):
        if coset[i] == index: 
          number += 1
        i += 1
      
      coset_freq.append( (number)/(len(coset)) )
      

    differences.append(find_total_difference(coset_freq, eng_freq))

    i = 0
    while i < len(alpha) - 1:
      coset_freq = rotate_list(coset_freq)
      differences.append(find_total_difference(coset_freq, eng_freq))
      i += 1 

    firstletter = 0
    secondletter = 1
    
    i = 0
    while i < len(differences) - 1:
      if (differences[i] < differences[firstletter]):
        secondletter = firstletter
        firstletter = i
        
      elif (differences[i] < differences[secondletter]):
        secondletter = i

      i += 1
    
    letter1 = alpha[firstletter]
    letter2 = alpha[secondletter]

    #return statement provided.  feel free to replace "letter1" and "letter2" with other variable names.
    return "the most likely letter is: " + letter1 + " followed by: " + letter2

def crack(ciphertext, alpha, eng_freq):  #Code provided
    """User-friendly walkthrough of decoding methods"""
    print("Your cipher text is: " + ciphertext)
    out = run_key_tests(ciphertext, alpha)
    print(out)
    x = int(input("Choose the key length you'd like to try: "))
    cosets = make_cosets(ciphertext, x)
    for index in range(len(cosets)):
        print("For coset " + str(index + 1) + ", " + find_likely_letters(cosets[index], alpha, eng_freq) + ".")
    s = input("Type the key you would like to use to decipher: ")
    print(vigenere_decode(ciphertext, s, alpha))
    print()




alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
#alpha = "ABCDEFGHIJKLMNÑOPQRSTUVWXYZ"

eng_freq = [.0817, .0149, .0278, .0425, .1270, .0223, .0202, .0609, .0697, .0015, .0077, .0403, .0241, .0675, .0751,
            .0193, .0010, .0599, .0633, .0906, .0276, .0098, .0236, .0015, .0197, .0007]
#es_freq = [12.53,1.42,4.68,5.86,13.68,0.69,1.01,0.70,6.25,0.44,0.02,4.97,3.15,6.71,0.31,8.68,2.51,0.88,6.87,7.98,4.63,3.93,0.90,0.01,0.22,0.90,0.52 ]
#example = "UZRZEGNJENVLISEXRHLYPYEGTESBJHJCSBPTGDYFXXBHEEIFTCCHVRKPNHWXPCTUQTGDJHTBIPRFEMJCNHVTCFSAIIJENREGSALHXHWZWRZXGTTVWGDHTEYXISAGQTCJPRSIAPTUMGZALHXHHSOHPWCZLBRZTCBRGHCDIQIKTOAAEFTOPYEGTENRAIALNRXLPCEPYKGPNGPRQPIAKWXDCBZXGPDNRWXEIFZXGJLVOXAJTUEMBLNLQHGPWVPEQPIAXATYENVYJEUEI"

texto= open("Maestra.txt",'r',encoding="utf8").read().upper()
cifrado = texto.replace('\n', '')
cifrado =  cifrado.replace(' ','')
example =  cifrado
#TEST Code
"""
print (index_of_coincidence(example, alpha))
print (friedman_test(example, alpha))
print (kasiski_test(example))
print (run_key_tests(example, alpha))
print (find_total_difference( [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
print( make_cosets(example, 6))

"""

# For the example, the key is "PLANET" and the plaintext is:
# "For many years the known planets of our solar system were mercury, venus, earth, mars, jupiter, saturn, uranus, neputne, and pluto.  However, it is now true that many people think pluto should no longer be considered a named planet.  New planets are currently being discovered, and it is very likely that many more will be in the near future."

#Try this:
crack(example, alpha, eng_freq)





Your cipher text is: WUBEFIQLZURMVOFEHMYMWTIXCGTMPIFKRZUPMVOIRQMMWOZMPULMBNYVQQQMVMVJLEYMHFEFNZPSDLPPSDLPEVQMWCXYMDAVQEEFIQCAYTQOWCXYMWMSEMEFCFWYEYQETRLIQYCGMTWCWFBSMYFPLRXTQYEEXMRULUKSGWFPTLRQAERLUVPMVYQYCXTWFQLMTELSFJPQEHMOZCIWCIWFPZSLMAEZIQVLQMZVPPXAWCSMZMORVGVVQSZETRLQZPBJAZVQIYXEWWOICCGDWHQMMVOWSGNTJPFPPAYBIYBJUTWRLQKLLLMDPYVACDCFQNZPIFPPKSDVPTIDGXMQQVEBMQALKEZMGCVKUZKIZBZLIUAMMVZ
Friedman test gives: 3.2865463727460695 and Kasiski gives this as the most likely: 5
Choose the key length you'd like to try: 5
For coset 1, the most likely letter is: E followed by: R.
For coset 2, the most likely letter is: M followed by: Q.
For coset 3, the most likely letter is: I followed by: T.
For coset 4, the most likely letter is: L followed by: W.
For coset 5, the most likely letter is: Y followed by: X.
